In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/30.TecDoc'
else:
    print('fuck')

path_root

'.'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import shutil
directory_out = os.path.join(path_root, './file/4.link')
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests
import json

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

import time

# = = =  = = =  = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = =  = = =  = = =

directory_in = os.path.join(path_root, './file/3.make')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['Part Code'] = df_input['Part Code'].astype(int)
    df_input['Make No'] = df_input['Make No'].astype(int)
    df_input['Make Code'] = df_input['Make Code'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global api, list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            data = {'getArticleLinkedAllLinkingTarget4': {'articleCountry': 'DE',
                                                          'articleId': crawler_series['Part Code'],
                                                          'country': 'DE',
                                                          'countryGroupFlag': False,
                                                          'lang': 'qa',
                                                          'linkingTargetManuId': crawler_series['Make Code'],
                                                          'linkingTargetType': 'VOLB',
                                                          'provider': 23365,
                                                          'withMainArticles': False}}

            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_check = crawler_series['Check_Url']

                    resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                         data=json.dumps(data),
                                         headers=get_header(**{'X-Api-Key': api}),
                                         proxies=get_proxy(),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if dict_['status'] == 200:
                            break
                        elif dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                            print(f'fuck api {api}')
                            time.sleep(3)
                            with open('./api.txt', 'r') as api_txt:
                                api = api_txt.read()
                except requests.exceptions.ProxyError as e:
                    if '456 CN Client Forbidden' in str(e):
                        print('Long live the People\'s Republic of China!')
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            list_dict_link = dict_['data']['array'][0]['articleLinkages']['array']

            # = = =  = = =  = = =

            if not list_dict_link:
                raise

            # = = =  = = =  = = =

            list_dict_link_2, index_begin, index_end = [], 0, 25
            while True:
                dict_link_2 = {'array': []}
                if index_end < len(list_dict_link):
                    for dict_link in list_dict_link[index_begin:index_end]:
                        dict_link_2['array'].append({'articleLinkId': dict_link['articleLinkId'], 'linkingTargetId': dict_link['linkingTargetId']})
                    list_dict_link_2.append(dict_link_2)
                    index_begin, index_end = index_end, index_end + 25
                else:
                    for dict_link in list_dict_link[index_begin:]:
                        dict_link_2['array'].append({'articleLinkId': dict_link['articleLinkId'], 'linkingTargetId': dict_link['linkingTargetId']})
                    list_dict_link_2.append(dict_link_2)
                    break

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.extend([{'Part Code': crawler_series['Part Code'],
                                           'Make No': crawler_series['Make No'],
                                           'Make': crawler_series['Make'],
                                           'Make Code': crawler_series['Make Code'],
                                           'JOIN_PCMC': crawler_series['JOIN_PCMC'],
                                           'Link No': i + 1,
                                           'Json_Link': json.dumps(dict_link_2),
                                           'JOIN_PCMCLN': f'''{crawler_series['JOIN_PCMC']};{i + 1}''',
                                           'Check_Url': crawler_series['Check_Url']} for i, dict_link_2 in enumerate(list_dict_link_2)])

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['JOIN_PCMC']}. {url_check}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make No', 'Link No'],
                                                                 ascending=[True, True, True],
                                                                 ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''./{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Code', 'Make'],
                                                               ascending=[True, True],
                                                               ignore_index=True)
        df_error.to_excel(os.path.join(path_root, f'''./file/{file.removesuffix('.xlsx')}-link_error.xlsx'''), index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求1次] - [剩余29条] - [crawler_2（100.00%）] - [2024/12/04 18:01:42] - 936842594;16. https://web.tecalliance.net/tecdoc/qa/parts/30/3 397 110 587/detail?groups=298

[True] - [请求1次] - [剩余28条] - [crawler_2（100.00%）] - [2024/12/04 18:01:42] - 427702909;882. https://web.tecalliance.net/tecdoc/qa/parts/30/3 397 009 110/detail?groups=298

[True] - [请求1次] - [剩余27条] - [crawler_2（100.00%）] - [2024/12/04 18:01:42] - 509591881;92. https://web.tecalliance.net/tecdoc/qa/parts/30/3 397 014 621/detail?groups=298

[True] - [请求1次] - [剩余26条] - [crawler_2（100.00%）] - [2024/12/04 18:01:42] - 775289383;111. https://web.tecalliance.net/tecdoc/qa/parts/30/3 397 014 637/detail?groups=298

[True] - [请求1次] - [剩余25条] - [crawler_2（100.00%）] - [2024/12/04 18:01:42] - 1104746029;2. https://web.tecalliance.net/tecdoc/qa/parts/30/3 398 112 213/detail?groups=298

[True] - [请求1次] - [剩余24条] - [crawler_2（100.00%）] - [2024/12/04 18:01:42] - 996374567;2243. https://web.tecalliance.net/tecdoc/qa/parts/30/3 397 015 000/d